# **POLITICAL QUESTION-ANSWERING**

# **Installation + Set-Up**

In [ ]:
!pip install tokenizers
!pip install transformers

import os
import tweepy as tw
import pandas as pd
from google.colab import files
import numpy as np
import re
import json
import string
from transformers import BertTokenizer, TFBertForQuestionAnswering
import torch
import tensorflow as tf
import nltk

     |████████████████████████████████| 3.3MB 3.9MB/s 
     |████████████████████████████████| 2.3MB 4.0MB/s 
     |████████████████████████████████| 901kB 38.3MB/s 


In [ ]:
import random

In [ ]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
from nltk.tokenize import RegexpTokenizer

In [ ]:
from transformers import pipeline
from transformers import BertForQuestionAnswering, AutoTokenizer

In [ ]:
consumer_key= '7mRsqhYU5DXe4oVA8m8u6Q6mE'
consumer_secret= 'vAYh40un148gp8H7mxFrpDfGe04Wzwda7Qpf0qUBL6Sgm0u72o'
access_token= '1189705328381906944-XNTFRajHw9n50A90sDTz63BV8iTsSi'
access_token_secret= '3ru03Pk4g0QpfliJoxCALUkjYfl7oX7WH5SRcDQetljQf'

In [ ]:
# Fetching data file from computer
uploaded = files.upload()

Saving Politician User Data - Sheet1.csv to Politician User Data - Sheet1.csv


In [ ]:
uploaded_old = files.upload()

Saving out.csv to out.csv


# **Politician Class**

In [ ]:
class Politician:
  def __init__(self, name, twitteracc1, twitteracc2, state, party, pos, tweets, idx):
    self.name = name
    self.twitteracc1 = twitteracc1
    self.twitteracc2 = twitteracc2
    self.state = state
    self.party = party 
    self.pos = pos
    self.tweets = tweets
    self.idx = idx

# **Twitter API:** Retrieving + Initializing Politician Data

This step only needs to run once, in order to call the Twitter API endpoint and initialize a new dataframe and CSV file with the retrieved Tweets that correspond to the politician names and Twitter handles that I manually inputted into a spreadsheet. 

In [ ]:
df0 = pd.read_csv('Politician User Data - Sheet1.csv')
df = df0.replace(np.nan, '', regex=True)

all_politicians = []

for idx, row in df.iterrows():
  d = row.to_dict()
  politician = Politician(d['Politician'], d['Account 1'], d['Account 2'], d['State'], d['State'], d['Position'], "", idx)
  all_politicians.append(politician)

In [ ]:
# Weird formatting error for only 1 of the entries -> manual fix
all_politicians[102].name = "Pence, Mike"

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
# ONLY RUN THIS ONCE TO GET TWITTER DATA, next time run the block below. 

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

tokenizer = RegexpTokenizer(r'\w+')
nltk_vocab = set(nltk.corpus.words.words())

for pol in all_politicians:
  try:
    tweets = api.user_timeline(screen_name = pol.twitteracc1, count=200, include_rts = False, tweet_mode = 'extended')
    for tweet in tweets[:len(tweets)]:
      tweets_no_nonenglish = " ".join(w for w in nltk.wordpunct_tokenize(tweet.full_text) if w.lower() in nltk_vocab or not w.isalpha() or not w.isdigit())
      tweet_words = tokenizer.tokenize(tweets_no_nonenglish)
      pol.tweets.update(tweet_words)
  except:
    try:
      tweets = api.user_timeline(screen_name = pol.twitteracc2, count=200, include_rts = False, tweet_mode = 'extended')
      for tweet in tweets[:len(tweets)]:
        tweets_no_nonenglish = " ".join(w for w in nltk.wordpunct_tokenize(tweet.full_text) if w.lower() in nltk_vocab or not w.isalpha() or not w.isdigit())
        tweet_words = tokenizer.tokenize(tweets_no_nonenglish)
        pol.tweets.update(tweet_words)
    except:
      print("No accounts found for politician " + pol.name)
  

# **Training Set:** SQuAD 2.0 

In [ ]:
!wget https://p4-squad-data.s3.us-east-2.amazonaws.com/train-v2.0.json

--2021-05-22 21:47:16--  https://p4-squad-data.s3.us-east-2.amazonaws.com/train-v2.0.json
Resolving p4-squad-data.s3.us-east-2.amazonaws.com (p4-squad-data.s3.us-east-2.amazonaws.com)... 52.219.106.130
Connecting to p4-squad-data.s3.us-east-2.amazonaws.com (p4-squad-data.s3.us-east-2.amazonaws.com)|52.219.106.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  63.5MB/s    in 0.6s    

2021-05-22 21:47:17 (63.5 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



# **Testing Set:** Politician Tweets

In [ ]:
df0_1 = pd.read_csv('out.csv')
df1 = df0_1.replace(np.nan, '', regex=True)
print(df1)

all_politicians = []

for idx, row in df1.iterrows():
  d = row.to_dict()
  politician = Politician(d['0'], d['1'], d['2'], d['3'], d['4'], d['5'], d['6'], idx)
  all_politicians.append(politician)

                       0  ...                                                  6
0         Baldwin, Tammy  ...   Looking forward to it! \n \nHint: Squeaky vs....
1         Barrasso, John  ...   .@JoeBiden’s “infrastructure” bill is a slush...
2        Bennet, Michael  ...   It's unacceptable that there is still a gende...
3      Blackburn, Marsha  ...   Blue states can’t stop mooching off the feder...
4    Blumenthal, Richard  ...   There is real concern that cruises may premat...
..                   ...  ...                                                ...
98            Wyden, Ron  ...   A fire warning. In Oregon. In April. The clim...
99           Young, Todd  ...   Our TRUST Act would set in motion a bipartisa...
100           Biden, Joe  ...   The American Jobs Plan is the largest America...
101       Harris, Kamala  ...   Help us spread the word: A special enrollment...
102          Pence, Mike  ...   Our deepest sympathies and prayers to Her Maj...

[103 rows x 7 columns]


In [ ]:
print(all_politicians[0].tweets)

 Looking forward to it! 
 
Hint: Squeaky vs. fried cheese curds is on my list of topics. https://t.co/9QEw8iZYs0 I support the bipartisan #CORPSAct because it's a bold plan that:

✅ Empowers more Americans to serve their communities
✅ Increases compensation &amp; education assistance
✅ Supports local, community-driven service initiatives

Congress must take action now! https://t.co/rziTOu9sFL Wisconsin will be receiving over $579 million in #AmericanRescuePlan funding to help early childhood educators and family child care providers keep their doors open, and help parents, especially mothers, get back to work. #HelpIsHere https://t.co/pQg2u2kZpj Thanks to @VoiceOfMilk for your support in our push for the #DairyPride Act. https://t.co/KB2cnrcpkC I’ve joined @SenBobCasey and @SenStabenow to intro the Junk Plan Accountability and Disclosure Act because we should protect people so they don’t get stuck paying thousands of dollars out of pocket when they need health care. https://t.co/xaOp6V

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
nltk_vocab = set(nltk.corpus.words.words())
all_politicians_untokeniz = []

for idx,pol in enumerate(all_politicians):
  curr_tweets = pol.tweets
  all_politicians_untokeniz.append(curr_tweets)
  if isinstance(curr_tweets, str):
      tweets_no_nonenglish = " ".join(w for w in nltk.wordpunct_tokenize(curr_tweets) if w.lower() in nltk_vocab or not w.isalpha() or not w.isdigit())
      tweets_manual_tokeniz = ""
      for word in tweets_no_nonenglish.split(" "):
        # Manually removing hyperlinks embedded in tweets. 
        if "http" not in word:
          tweets_manual_tokeniz = tweets_manual_tokeniz + " " + word
      tweet_words = tokenizer.tokenize(tweets_manual_tokeniz)
  else:
    tweet_words = curr_tweets
  
  pol.tweets = tweet_words

In [ ]:
print(all_politicians[0].tweets)

['Looking', 'forward', 'to', 'it', 'Hint', 'Squeaky', 'vs', 'fried', 'cheese', 'curds', 'is', 'on', 'my', 'list', 'of', 'topics', 't', 'co', '9QEw8iZYs0', 'I', 'support', 'the', 'bipartisan', 'CORPSAct', 'because', 'it', 's', 'a', 'bold', 'plan', 'that', 'Empowers', 'more', 'Americans', 'to', 'serve', 'their', 'communities', 'Increases', 'compensation', 'amp', 'education', 'assistance', 'Supports', 'local', 'community', 'driven', 'service', 'initiatives', 'Congress', 'must', 'take', 'action', 'now', 't', 'co', 'rziTOu9sFL', 'Wisconsin', 'will', 'be', 'receiving', 'over', '579', 'million', 'in', 'AmericanRescuePlan', 'funding', 'to', 'help', 'early', 'childhood', 'educators', 'and', 'family', 'child', 'care', 'providers', 'keep', 'their', 'doors', 'open', 'and', 'help', 'parents', 'especially', 'mothers', 'get', 'back', 'to', 'work', 'HelpIsHere', 't', 'co', 'pQg2u2kZpj', 'Thanks', 'to', 'VoiceOfMilk', 'for', 'your', 'support', 'in', 'our', 'push', 'for', 'the', 'DairyPride', 'Act', 't'

In [ ]:
print(all_politicians[102].name)
print(all_politicians_untokeniz[102])

Pence, Mike
 Our deepest sympathies and prayers to Her Majesty The Queen, the Royal Family and all the people of our great ally the United Kingdom. God bless Prince Philip, a man of dignity and devotion to family and country. https://t.co/2RPrwmBYAx https://t.co/D22adkH20A Excited to launch @AmericanFreedom, an advocacy organization that will defend and build upon the successful policies of the Trump-Pence Administration. AAF will help defeat the Radical Left Agenda and Advance American Freedom for generations to come! 🇺🇸 https://t.co/tnR04lTCXb “Do not be afraid, for I know that you are looking for Jesus, who was crucified. He is not here; He has risen, just as He said.” (Matthew 28) Karen and I send our best wishes for a Blessed Easter to all who greet the Empty Tomb with Full Hearts. 
He is Risen! #HappyEaster https://t.co/tsdh1WPBni https://t.co/Pg1ATtKXui Our hearts are with the family of US Capitol Police Officer William “Billy”Evans who gave his life protecting our Capitol today

In [ ]:
names = [pol.name for pol in all_politicians]
tw1 = [pol.twitteracc1 for pol in all_politicians]
tw2 = [pol.twitteracc2 for pol in all_politicians]
state = [pol.state for pol in all_politicians]
party = [pol.party for pol in all_politicians]
pos = [pol.pos for pol in all_politicians]
tweets_total = [pol.tweets for pol in all_politicians]

df_to_save = pd.DataFrame([names, tw1, tw2, state, party, pos, tweets_total]).transpose()

In [ ]:
# ONLY RUN FIRST TIME. 

compression_opts = dict(method='zip', archive_name='out.csv') 
df_to_save.to_csv('out.zip', index=False, compression=compression_opts)  

# **MODEL 1:** BERT for Question Answering

In [ ]:
name_to_info = {}
for pol in all_politicians:
  name = pol.name.split(", ")
  if not name[0] in name_to_info:
    name_to_info[name[0]] = pol
  else:
    # If there are two politicians with the last name, store first and last name. 
    temp = name_to_info[name[0]].name 
    temp_pol = name_to_info[name[0]]
    del name_to_info[name[0]]
    name_to_info[temp.split(", ")[1] + " " + temp.split(", ")[0]] = temp_pol
    name_to_info[pol.name.split(", ")[1] + " " + pol.name.split(", ")[0]] = pol

In [ ]:
train_set, validation_set, all_text, unanswerable = [[] for _ in range(4)]

for pol in all_politicians:
  for tweet in pol.tweets:
    if random.randint(1,10) is 1: 
      validation_set.append((pol, tweet))
    else:
      train_set.append((pol, tweet))

In [ ]:
modelname = 'deepset/bert-base-cased-squad2'

model = BertForQuestionAnswering.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)

In [ ]:
# Format: "What does " + name + "think about " + question input. 
def generate_answer(name, query, model):
  name_lst = name.split(" ")
  context = ""

  if name in name_to_info:
    context = " ".join(name_to_info[name].tweets)
  else:
    name_lst = name.split(" ")
    if name_lst[0] in name_to_info:
      context = " ".join(name_to_info[name_lst[0]].tweets)
    elif name_lst[1] in name_to_info:
      context = " ".join(name_to_info[name_lst[1]].tweets)
    
  question = "What do I think about " + query

  output = model({'question': question,'context': context})
  percent = output['score']
  answer = output['answer']

  return percent, answer

In [ ]:
# Examples
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)
percent, answer = generate_answer("Biden", "pandemic", nlp)

In [ ]:
print("Answer: '" + answer + "' with accuracy " + str(np.round(percent, 2)*100) + "%")

Answer: 'we cannot let people go hungry' with accuracy 45.0%


# **MODEL 2**: Cosine Similarity Text Retrieval

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math
import re
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.corpus import wordnet as guru

In [ ]:
name_to_sent_tok = {}

for name,pol in name_to_info.items():
  original_tweets = all_politicians_untokeniz[pol.idx]
  sentences = str.splitlines(original_tweets)

  all_sents = []
  sents_tok = []
  for sent in sentences:
    s_lst = word_tokenize(sent)
    stopwrds = set(["co", "https", "amp", "#"]) #remove weird urls 
    sent = ""
    for i,word in enumerate(s_lst):
      if word is "." or word is "?" or word is "!":
        sent = sent + word 
        sents_tok.append(sent)
        sent = ""
      else:
        if (not "co/" in word) and (not "https" in word):
          if (sent == ""):
            sent = word
          else:
            sent = sent + " " + word

  name_to_sent_tok[name] = sents_tok

In [ ]:
def sent_to_vec(sent):
  regex = re.compile(r"\w+")
  words = regex.findall(sent)
  return Counter(words)

In [ ]:
def cos_similarity(v1, v2):
  intersection = set(np.intersect1d(list(v1.keys()), list(v2.keys())))
  numrtr = 0
  for i in intersection:
    numrtr += v1[i] * v2[i]

  x_sum, y_sum = [0 for _ in range(2)]
  for x in list(v1.keys()):
    x_sum += v1[x] ** 2
  for y in list(v2.keys()):
    y_sum += v2[y] ** 2

  denom = math.sqrt(x_sum) * math.sqrt(y_sum)

  if not denom:
    return 0.0
  else:
    return float(numrtr) / denom

In [ ]:
# Format: "What does " + name + "think about " + question input. 
def generate_answer_cs(name, query):
  name_lst = name.split(" ")
  context = ""
  input_n = ""

  if name in name_to_sent_tok :
    input_n = name
  else:
    name_lst = name.split(" ")
    if name_lst[0] in name_to_sent_tok :
      input_n = name_lst[0]
    elif name_lst[1] in name_to_sent_tok :
      input_n = name_lst[1]

  q_lst = word_tokenize(query)
  stopwrds = set(stopwords.words('english'))
  q_tok = {word for word in q_lst if not word in stopwrds}
    
  all_sentences = name_to_sent_tok[input_n]

  max_c_sim = 0
  max_tweet = ""
  for tweet in all_sentences:
    q_vec = sent_to_vec(query)
    t_vec = sent_to_vec(tweet)

    cos_sim = cos_similarity(q_vec, t_vec)
    if cos_sim > max_c_sim:
      max_tweet = tweet 
    
  return max_tweet

In [ ]:
print(generate_answer_cs("Biden", "pandemic"))

Together we ’ ll change the course of the pandemic , build a bridge toward economic recovery , and invest in racial justice.


# Evaluation + Performance

In [ ]:
import time
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import plotly.express as px
from sklearn import metrics
from sklearn import feature_extraction
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Tokenizing all of the response words
tokenizer = RegexpTokenizer(r'\w+') # getting rid of punctuation.
all_text = ""
all_words = []
for pol in all_politicians_untokeniz:
  all_text = all_text + " " + pol

all_words = tokenizer.tokenize(all_text) # splitting up all of the words.
stop_words = set(stopwords.words('english')) # removing stop words (most common words in English language, like 'and' or 'the').
stop_words.update(["co", "https", "amp", "today", "1", "u", "19", "one", "us"]) #removing common words that don't aid in understanding data.

# Finding a frequency distribution of words and ngrams
lowercase = [w.lower() for w in all_words]
word_distribution = nltk.FreqDist(w for w in lowercase if not w in stop_words)
top_words = word_distribution.most_common(50)

ngrams = nltk.ngrams(all_words, 2)
ngrams_no_stop_words = [gram for gram in ngrams if not any(stop in gram for stop in stop_words)] #removing stop words
ngram_distribution = nltk.FreqDist(ngrams_no_stop_words)
top_ngrams = ngram_distribution.most_common(25)

In [ ]:
words = [w for w,c in top_words]
counts = [c for w,c in top_words]

fig = px.bar(x=words, y=counts, labels={'x':'Words', 'y':'Word Frequencies'})
fig.show()

In [ ]:
start = time.time()
print("hello")
end = time.time()
print(end - start)

hello
0.00013184547424316406


In [ ]:
val_uploaded = files.upload()

Saving Political Q-A Eval - Sheet1.csv to Political Q-A Eval - Sheet1.csv


In [ ]:
# Retrieving and setting up custom-made validation data

df_val = pd.read_csv('Political Q-A Eval - Sheet1.csv')
df_val0 = df_val.replace(np.nan, '', regex=True)
dfval = df_val.replace(np.nan, '', regex=True).T
print(dfval)

pol_names = df_val0.columns.to_list()[1:]

all_val = []
for pol in pol_names:
  all_val.append((pol, {}))
 
for i, (idx, row) in enumerate(dfval.iterrows()):
  d = row.to_dict()
  if i == 0:
    for x in range(0,25):
      for pol in range(0,len(all_val)):
        all_val[pol][1][d[x]] = ""
  else:
    for j, (k, v) in enumerate(all_val[i-1][1].items()):
      all_val[i-1][1][k] = d[j]

print(all_val)

                                                           0   ...                                                 25
Unnamed: 0                                         Republican  ...                                      China / Asian
Baldwin     Republicans in Wisconsin and Washington want t...  ...                                                   
Barrasso                        Economy is quickly recovering  ...                                                   
Bennet      Senate Republicans can’t give up promoting Tru...  ...                                                   
Blackburn                                                      ...  Keep China Our of Solar Energy Act. #Communist...
Blumenthal                                                     ...  Anti-Asian hate crimes have increased by 150% ...
Blunt       Democrat and Republican administrations alike ...  ...                                                   
Booker      History won't be kind to the 43 Senate Repub

In [ ]:
bert_incorrect, cs_incorrect, bert_correct, cs_correct = [[] for _ in range(4)]
bert_avg_f1, cs_avg_f1, bert_avg_acc, cs_avg_acc, bert_avg_time, cs_avg_time = [0 for _ in range(6)]
total = 0

for pol, pol_dict in all_val:
  for topic, correct_resp in pol_dict.items():

    bert_start = time.time()
    bert_percent, bert_resp = generate_answer(pol, topic, nlp)
    bert_end = time.time()
    bert_time = bert_end - bert_start

    cs_start = time.time()
    cs_resp = generate_answer_cs(pol, topic)
    cs_end = time.time()
    cs_time = cs_end - cs_start

    y_true = [correct_resp.split(" ")]
    bert_pred = [bert_resp.split(" ")]
    cs_pred = [cs_resp.split(" ")]

    binarizer = MultiLabelBinarizer()
    binarizer.fit(y_true)

    bert_f1 = metrics.f1_score(binarizer.transform(y_true), binarizer.transform(bert_pred), average='macro')
    cs_f1 = metrics.f1_score(binarizer.transform(y_true), binarizer.transform(cs_pred), average='macro')
    bert_accuracy = metrics.accuracy_score(binarizer.transform(y_true), binarizer.transform(bert_pred))
    cs_accuracy = metrics.accuracy_score(binarizer.transform(y_true), binarizer.transform(cs_pred))
    
    if bert_f1 <= 0.25:
      bert_incorrect.append({'Correct': correct_resp, 'Bert': bert_resp, 'F1': bert_f1, 'Accuracy': bert_accuracy, 'Bert Acc': bert_percent, 'Time': bert_time})
    else:
      bert_correct.append({'Correct': correct_resp, 'Bert': bert_resp, 'F1': bert_f1, 'Accuracy': bert_accuracy, 'Bert Acc': bert_percent, 'Time': bert_time})

    if cs_f1 <= 0.25:
      cs_incorrect.append({'Correct': correct_resp, 'CS': cs_resp, 'F1': cs_f1, 'Accuracy': cs_accuracy, 'Time': cs_time})
    else:
      cs_correct.append({'Correct': correct_resp, 'CS': cs_resp, 'F1': cs_f1, 'Accuracy': cs_accuracy, 'Time': cs_time})
    
    bert_avg_f1 += bert_f1
    bert_avg_acc += bert_accuracy
    cs_avg_f1 += cs_f1
    cs_avg_acc += cs_accuracy
    bert_avg_time += bert_time
    cs_avg_time += cs_time

    total += 1

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning:

unknown class(es) ['Americans', 'and', 'generations', 'has', 'impacted', 'millions', 'of', 'unborn'] will be ignored

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning:

unknown class(es) ['I', 'am', 'any', 'free', 'is', 'not', 'unfree', 'while', 'woman'] will be ignored

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning:

unknown class(es) ['ending', 'of', 'poverty', 'the', 'violence'] will be ignored

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning:

unknown class(es) ['#', ',', ';', 'BlackHistoryMonth', 'Celebrating', 'Hughes', 'Langston', 'ancestors', 'as', 'birthday', 'celebrate', 'day', 'first', 'his', 'history', 'honor', 'in', 'indelible', 'kick', 'of', 'off', 'poetic', 'the', 'today', 'urgent', 'we', 'wisdom'] will be ignored

/usr/local/lib/python3.7/dist-packages/sklearn/pr

In [ ]:
bert_avg_f1 /= total
cs_avg_f1 /= total
bert_avg_accuracy /= total
cs_avg_accuracy /= total
bert_avg_time /= total
cs_avg_time /= total

# **TODO**

1. Add Bert to frontend
2. Add synonyms, remove stopwords for second model
3. Model comparisons and analysis